In [17]:
import math

import numpy as np

from generate_nerf_data_utils import read_ts, ts_to_poses

In [18]:
ts_path = "pen_ol.ts"

In [19]:
ts = read_ts(ts_path)
poses = ts_to_poses(ts)

In [20]:
def create_linear_transform(x_translation, y_translation, z_translation, x_rotation, y_rotation, z_rotation):
    """Creates a linear transformation from the given parameters.

    Args:
        x_translation (float): The x translation.
        y_translation (float): The y translation.
        z_translation (float): The z translation.
        x_rotation (float): The x rotation, in degrees.
        y_rotation (float): The y rotation, in degrees.
        z_rotation (float): The z rotation, in degrees.

    Returns:
        np.ndarray: 4x4 standard homogenous transformation matrix.
    """

    # Convert degrees to radians.
    x_rotation = math.radians(x_rotation)
    y_rotation = math.radians(y_rotation)
    z_rotation = math.radians(z_rotation)

    # Create the rotation matrix.
    rotation_matrix_x = np.array([
        [1, 0, 0, 0],
        [0, math.cos(x_rotation), -math.sin(x_rotation), 0],
        [0, math.sin(x_rotation), math.cos(x_rotation), 0],
        [0, 0, 0, 1]
    ])
    rotation_matrix_y = np.array([
        [math.cos(y_rotation), 0, math.sin(y_rotation), 0],
        [0, 1, 0, 0],
        [-math.sin(y_rotation), 0, math.cos(y_rotation), 0],
        [0, 0, 0, 1]
    ])
    rotation_matrix_z = np.array([
        [math.cos(z_rotation), -math.sin(z_rotation), 0, 0],
        [math.sin(z_rotation), math.cos(z_rotation), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    rotation_matrix = rotation_matrix_x @ rotation_matrix_y @ rotation_matrix_z

    # Create the translation matrix.
    translation_matrix = np.array([
        [1, 0, 0, x_translation],
        [0, 1, 0, y_translation],
        [0, 0, 1, z_translation],
        [0, 0, 0, 1]
    ])

    # Combine the rotation and translation matrices.
    return translation_matrix @ rotation_matrix


In [21]:
print(f"Shape of poses: {poses.shape}")

Shape of poses: (1931, 4, 4)


In [22]:
transform = create_linear_transform(0, 0, 0, 0, 30, 180)

In [23]:
# Apply the transform to each pose.
poses = np.transpose(np.transpose(poses, (0, 2, 1)) @ transform.T, (0, 2, 1))

In [24]:
poses = np.transpose(poses, (0, 2, 1))
poses = poses.reshape((poses.shape[0], -1))

with open(ts_path.replace(".ts", "_corrected.csv"), "w") as f:
    for i in range(poses.shape[0]):
        f.write(",".join([str(x) for x in poses[i]]) + "\n")